In [2]:
from utils import *

import pandas as pd
import numpy as np
from pycaret.classification import *
from sklearn.impute import KNNImputer
from sklearn.preprocessing import *

In [7]:
# Tengo que crear una función que recoja todas las modificaciónes que hemos hecho hasta ahora para implementarla en el df objetivo
# Las ejecuto EXACTAMENTE en el orden en el que las hice

def tratamiento(df):
    # V3
    df2 = df.copy() 
    
    from_zero_to_nan = ['funder','gps_height','installer','longitude','construction_year']
    for fn in from_zero_to_nan:
        a = df2[fn].isnull().sum()
        df2[fn].replace(('0', 0), (np.nan), inplace = True)
        b = df2[fn].isnull().sum()
        del a
        del b

    #lista_col = df2.columns.tolist() # para que lo hice?????? ####################

    df2.loc[df2['gps_height'] < 0, 'gps_height'] = np.nan # Para alturas negativas

    # parece que hay 1812 registros con -2.000000e-08. Los ponemos como missings
    df2['latitude'].replace(-2.000000e-08, np.nan, inplace = True)  

    df3 = df2.copy()    # Estoy copiando y pegando las lineas en las que modificaba los df en los otros jupiter
    del df2             # para que no ocupe memoria

    # V3.1
    del df3['recorded_by']
    del df3['extraction_type']
    del df3['extraction_type_group']
    del df3['scheme_name']
    del df3['management']

    df3['management_group'].replace(['other', 'unknown'], 'other/unknown', inplace = True)
    df3['payment_type'].replace(['other', 'unknown'], 'other/unknown', inplace = True)

    del df3['payment']
    del df3['water_quality']
    del df3['quantity_group']

    df3['quantity'].replace('unknown', np.nan, inplace = True)

    del df3['source']
    df3['source_class']. replace('unknown', np.nan, inplace = True)

    del df3['waterpoint_type_group']
    
    # df3 real contiene status_group

    # V4.1
    df4 = df3.copy()
    del df3

    df4['date_recorded'] = pd.to_datetime(df4['date_recorded'])
    df4['year_recorded'] = df4['date_recorded'].dt.year
    del df4['date_recorded']

    df4_distr_aleatoria = df4.copy()
    df4_distr_aleatoria['funder'].replace('unknown', np.nan, inplace=True)
    imputer_aleatorio = fe_imp.RandomSampleImputer()
    df4_distr_aleatoria['funder'] = imputer_aleatorio.fit_transform(df4_distr_aleatoria[['funder']])

    df4_distr_aleatoria['installer'].replace('unknown', np.nan, inplace=True)
    imputer_aleatorio = fe_imp.RandomSampleImputer()
    df4_distr_aleatoria['installer'] = imputer_aleatorio.fit_transform(df4_distr_aleatoria[['installer']])

    df4_Distinct = df4_distr_aleatoria.copy()
    df4_Distinct['funder'].replace('unknown', np.nan, inplace=True)
    missing_count = df4_Distinct['funder'].isnull().sum()
    random_strings = [''.join(np.random.choice(list('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'), size=4)) for _ in range(missing_count)]
    df4_Distinct.loc[df4_Distinct['funder'].isnull(), 'funder'] = random_strings

    df4_Distinct['installer'].replace('unknown', np.nan, inplace=True)
    missing_count = df4_Distinct['installer'].isnull().sum()
    random_strings = [''.join(np.random.choice(list('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'), size=4)) for _ in range(missing_count)]
    df4_Distinct.loc[df4_Distinct['installer'].isnull(), 'installer'] = random_strings

    df5 = df4_Distinct.copy()

    # V4.2
    df5_geo = df5.copy()
    df5_geo = df5.loc[:, ['id', 'longitude', 'latitude', 'wpt_name', 
        'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga', 'ward']]
    
    imputer_knn = KNNImputer(n_neighbors=3)
    columns_to_impute = ['longitude', 'latitude']
    df5_geo_subset = df5_geo[columns_to_impute]
    imputer_knn.fit(df5_geo_subset)
    df5_geo_subset_imputed = pd.DataFrame(imputer_knn.transform(df5_geo_subset), columns=columns_to_impute)
    df5_geo_imputed = df5_geo.copy()
    df5_geo_imputed[columns_to_impute] = df5_geo_subset_imputed
    df5_geo_imputed

    df5_altura = df5_geo_imputed.loc[:, ['id', 'longitude', 'latitude']].copy()
    df5_altura = pd.merge(df5_altura, df5[['id', 'gps_height']], on='id', how='inner')

    imputer_knn = KNNImputer(n_neighbors=2) 
    imputer_knn.fit(df5_altura)
    df5_altura_imputed = pd.DataFrame(imputer_knn.transform(df5_altura), columns=df5_altura.columns)


    column_order = df5.columns.tolist()
    df5 = df5.drop(['gps_height', 'longitude', 'latitude'], axis=1)
    df5.head()
    df5 = pd.merge(df5, df5_altura_imputed, on='id', how='inner')
    df5 = df5[column_order]

    df5_sin_missing_location = df5.copy()
    missing_count = df5_sin_missing_location['subvillage'].isnull().sum()
    random_strings = [''.join(np.random.choice(list('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'), size=4)) for _ in range(missing_count)]
    df5_sin_missing_location.loc[df5_sin_missing_location['subvillage'].isnull(), 'subvillage'] = random_strings

    df5_sin_missing_location['wpt_name'].replace('none', np.nan, inplace=True)
    missing_count = df5_sin_missing_location['wpt_name'].isnull().sum()
    random_strings = [''.join(np.random.choice(list('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'), size=4)) for _ in range(missing_count)]
    df5_sin_missing_location.loc[df5_sin_missing_location['wpt_name'].isnull(), 'wpt_name'] = random_strings

    # V4.3
    df5 = df5_sin_missing_location.copy()
    del df5_sin_missing_location

    df5_public_meeting = df5.copy()
    imputer_aleatorio = fe_imp.RandomSampleImputer()
    df5_public_meeting['public_meeting'] = imputer_aleatorio.fit_transform(df5_public_meeting[['public_meeting']])
    del df5

    # V4.4
    df5 = df5_public_meeting.copy()
    del df5_public_meeting
    df5['scheme_management'] = df5['scheme_management'].replace(np.nan, 'Other')

    df5_permit = df5.copy()
    imputer_aleatorio = fe_imp.RandomSampleImputer()
    df5_permit['permit'] = imputer_aleatorio.fit_transform(df5_permit[['permit']])

    df6 = df5_permit.copy()
    imputer_aleatorio = fe_imp.RandomSampleImputer()
    df6['source_class'] = imputer_aleatorio.fit_transform(df6[['source_class']])

    imputer_aleatorio = fe_imp.RandomSampleImputer()
    df6['quantity'] = imputer_aleatorio.fit_transform(df6[['quantity']]) 

    del df5, df5_permit

    # V4.6
    # df6 = df6

    df6_construction_year = df6.copy()
    df6_construction_year = df6_construction_year[['id', 'wpt_name', 'subvillage', 'ward', 'construction_year']]
    df6_construction_year

    df6_construction_year_sin_missings = df6_construction_year.copy()
    df6_construction_year_solo_missings = df6_construction_year.copy()
    df6_construction_year_solo_missings = df6_construction_year_solo_missings[df6_construction_year_solo_missings['construction_year'].isna()]
    df6_construction_year_sin_missings = df6_construction_year.dropna(subset=['construction_year'])
    df6_construction_year_solo_missings

    df6_construction_year_sin_missings.info()

    variables_categoricas = ['wpt_name', 'subvillage', 'ward']

    le = LabelEncoder()
    df6_no_missings = df6_construction_year_sin_missings.copy()
    for variable in variables_categoricas:
        df6_no_missings[variable] = le.fit_transform(df6_no_missings[variable])
    df6_no_missings

    print("Para obtener construction_year")
    model_setup = setup(data=df6_no_missings, 
                        target = 'construction_year', 
                        session_id=1, 
                        )

    best_model = create_model('dt')

    df6_construction_year_solo_missings

    df6_construction_year_solo_missings_copy = df6_construction_year_solo_missings.copy()
    le = LabelEncoder()
    for variable in variables_categoricas:
        df6_construction_year_solo_missings_copy[variable] = le.fit_transform(df6_construction_year_solo_missings_copy[variable])

    df6_construction_year_solo_missings_copy2 = df6_construction_year_solo_missings_copy.drop('construction_year', axis=1)

    df6_construction_year_solo_missings_copy2 = predict_model(best_model, data=df6_construction_year_solo_missings_copy2)

    df6_construction_year_solo_missings_copy2 = df6_construction_year_solo_missings_copy2.rename(columns={'prediction_label': 'construction_year'})

    df6_construction_year_sin_missings = df6_construction_year_sin_missings.drop(['wpt_name', 'subvillage', 'ward'], axis = 1)

    df6_construction_year_solo_missings_copy2  = df6_construction_year_solo_missings_copy2.drop(['wpt_name', 'subvillage', 'ward', 'prediction_score'], axis = 1)

    df6_construction_year_sin_missings['construction_year'] = df6_construction_year_sin_missings['construction_year'].astype('int32')

    df6_construction_year_solo_missings_copy2['id'] = df6_construction_year_solo_missings_copy2['id'].astype('int64')

    df6_construction_year_sin_missings = pd.concat([df6_construction_year_sin_missings, df6_construction_year_solo_missings_copy2])

    df6 = df6.drop('construction_year', axis=1)

    df6 = pd.merge(df6, df6_construction_year_sin_missings, how = 'inner', on = 'id')

    column_order = ['id', 'amount_tsh', 'funder', 'gps_height', 'installer', 'longitude', 'latitude', 
                    'wpt_name', 'num_private', 'basin', 'subvillage', 'region', 'region_code', 'district_code',
                    'lga', 'ward', 'population', 'public_meeting', 'scheme_management', 'permit', 
                    'construction_year', 'extraction_type_class', 'management_group', 'payment_type', 
                    'quality_group', 'quantity', 'source_type', 'source_class', 'waterpoint_type']
    df6 = df6[column_order]
    return(df6)


<class 'pandas.core.frame.DataFrame'>
Index: 38691 entries, 0 to 59399
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 38691 non-null  int64  
 1   wpt_name           38691 non-null  object 
 2   subvillage         38691 non-null  object 
 3   ward               38691 non-null  object 
 4   construction_year  38691 non-null  float64
dtypes: float64(1), int64(1), object(3)
memory usage: 1.8+ MB
Para obtener construction_year


,Description,Value
0,Session id,1
1,Target,construction_year
2,Target type,Multiclass
3,Target mapping,"1960.0: 0, 1961.0: 1, 1962.0: 2, 1963.0: 3, 1964.0: 4, 1965.0: 5, 1966.0: 6, 1967.0: 7, 1968.0: 8, 1969.0: 9, 1970.0: 10, 1971.0: 11, 1972.0: 12, 1973.0: 13, 1974.0: 14, 1975.0: 15, 1976.0: 16, 1977.0: 17, 1978.0: 18, 1979.0: 19, 1980.0: 20, 1981.0: 21, 1982.0: 22, 1983.0: 23, 1984.0: 24, 1985.0: 25, 1986.0: 26, 1987.0: 27, 1988.0: 28, 1989.0: 29, 1990.0: 30, 1991.0: 31, 1992.0: 32, 1993.0: 33, 1994.0: 34, 1995.0: 35, 1996.0: 36, 1997.0: 37, 1998.0: 38, 1999.0: 39, 2000.0: 40, 2001.0: 41, 2002.0: 42, 2003.0: 43, 2004.0: 44, 2005.0: 45, 2006.0: 46, 2007.0: 47, 2008.0: 48, 2009.0: 49, 2010.0: 50, 2011.0: 51, 2012.0: 52, 2013.0: 53"
4,Original data shape,"(38691, 5)"
5,Transformed data shape,"(38691, 5)"
6,Transformed train set shape,"(27083, 5)"
7,Transformed test set shape,"(11608, 5)"
8,Numeric features,4
9,Preprocess,True


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.3917,0.0000,0.3917,0.3931,0.3908,0.3706,0.3706
1,0.4097,0.0000,0.4097,0.4116,0.4089,0.3896,0.3897
2,0.4157,0.0000,0.4157,0.4168,0.4144,0.3950,0.3951
3,0.3918,0.0000,0.3918,0.3964,0.3919,0.3706,0.3707
4,0.3955,0.0000,0.3955,0.3986,0.3954,0.3746,0.3746
5,0.4114,0.0000,0.4114,0.4165,0.4120,0.3915,0.3916
6,0.4047,0.0000,0.4047,0.4057,0.4039,0.3844,0.3845
7,0.4073,0.0000,0.4073,0.4084,0.4066,0.3867,0.3867
8,0.4040,0.0000,0.4040,0.4047,0.4024,0.3833,0.3834


(59400, 29)
                       Valores Nulos  % de Valores Nulos
id                                 0                 0.0
amount_tsh                         0                 0.0
funder                             0                 0.0
gps_height                         0                 0.0
installer                          0                 0.0
longitude                          0                 0.0
latitude                           0                 0.0
wpt_name                           0                 0.0
num_private                        0                 0.0
basin                              0                 0.0
subvillage                         0                 0.0
region                             0                 0.0
region_code                        0                 0.0
district_code                      0                 0.0
lga                                0                 0.0
ward                               0                 0.0
population         

In [ ]:
df6.to_pickle("./pickles_temp/V5_base.pkl")